In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
gc.collect()

官方指認欄位 = [2,6,10,12,13,15,18,19,21,22,25,26,36,37,39,48]
官方指認欄位 = [str(i) for i in 官方指認欄位]

df = pd.read_feather('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.feather')
test_data = pd.read_feather('../data/需預測的顧客名單及提交檔案範例.feather')
test_data.head()

,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26


# 根據apriori關聯分析演算法建立對應表,shop_tag映射到其他shop_tag的映射

In [4]:
from apyori import apriori
n = 5290
min_support = 0.1
min_confidence = 0.1
min_lift = 1.1
max_length = 2
data = []
for chid in tqdm(df.sample(n)['chid'].values):
    data.append(df.loc[df.chid==chid,'shop_tag'].value_counts().index.tolist())
association_rules = apriori(data, min_support=min_support, min_confidence=min_confidence, min_lift=min_lift,max_length=max_length) 
association_results = list(association_rules)
for item in association_results[:5]:
   pair = item[0] 
   items = [x for x in pair]
   print("Rule: " + items[0] + " -> " + items[1])
   print("Support: " + str(item[1]))
   print("Confidence: " + str(item[2][0][2]))
   print("Lift: " + str(item[2][0][3]))
   print("=====================================")

100%|██████████| 5290/5290 [02:10<00:00, 40.66it/s]

Rule: 10 -> 11
Support: 0.36843100189035916
Confidence: 0.4974476773864216
Lift: 1.1089330861248083
Rule: 10 -> 12
Support: 0.49206049149338377
Confidence: 0.6643695763144462
Lift: 1.1359130765040142
Rule: 10 -> 13
Support: 0.3270321361058601
Confidence: 0.44155181214905564
Lift: 1.2625995060910835
Rule: 14 -> 10
Support: 0.11871455576559546
Confidence: 0.16028586013272078
Lift: 1.1200953766209947
Rule: 10 -> 16
Support: 0.26767485822306236
Confidence: 0.3614088820826952
Lift: 1.1012978031206553


In [5]:
mapping = {}
for key in 官方指認欄位:
   mapping[key] = []
for item in association_results:
   pair = item[0] 
   items = [x for x in pair]
   source = items[0]
   target = items[1]
   if (source in 官方指認欄位) and (target in 官方指認欄位):
       mapping[source].append(target)
mapping

{'2': ['13', '22', '25', '39', '6'],
 '6': ['13', '19', '22', '25', '48'],
 '10': ['12', '13'],
 '12': ['13', '19', '22', '25', '26', '6'],
 '13': ['22', '48'],
 '15': [],
 '18': ['10', '12', '13', '19', '21', '22', '25', '26', '36', '39', '48', '6'],
 '19': ['13', '22'],
 '21': ['10', '12', '13', '19', '2', '22', '25', '26', '39', '48', '6'],
 '22': [],
 '25': ['13', '19', '22'],
 '26': ['13', '19', '22', '25', '48', '6'],
 '36': ['12', '13', '21', '22', '25', '48'],
 '37': [],
 '39': ['12', '13', '19', '22', '25', '26', '48', '6'],
 '48': []}

In [6]:
import joblib
joblib.dump(mapping,'../data/shop_tag_mapping.pkl')

['../data/shop_tag_mapping.pkl']

In [7]:
mapping = joblib.load('../data/shop_tag_mapping.pkl')
mapping

{'2': ['13', '22', '25', '39', '6'],
 '6': ['13', '19', '22', '25', '48'],
 '10': ['12', '13'],
 '12': ['13', '19', '22', '25', '26', '6'],
 '13': ['22', '48'],
 '15': [],
 '18': ['10', '12', '13', '19', '21', '22', '25', '26', '36', '39', '48', '6'],
 '19': ['13', '22'],
 '21': ['10', '12', '13', '19', '2', '22', '25', '26', '39', '48', '6'],
 '22': [],
 '25': ['13', '19', '22'],
 '26': ['13', '19', '22', '25', '48', '6'],
 '36': ['12', '13', '21', '22', '25', '48'],
 '37': [],
 '39': ['12', '13', '19', '22', '25', '26', '48', '6'],
 '48': []}

In [9]:
n = 100
start_dt = 24-6 # 取最近半年的數據
predicts = []
for chid in tqdm(test_data['chid'].values[:n]):
    a = df.loc[(df['chid']==chid)&(df['dt']>=start_dt),'shop_tag'].value_counts().to_frame() # 根據chid對shop_tag做value_counts()
    a['在指認欄位'] = False # 預設False
    if len(list(set(a.index)&set(官方指認欄位))) != 0: # 判斷這個chid出現過的shop_tag跟官方指認欄位是否有交集
        a.loc[list(set(a.index)&set(官方指認欄位)),'在指認欄位'] = True # 如果有交集則是True
    answer = a[a['在指認欄位']==True].index.tolist()[:3] # 取value_counts()前三大的當作答案
    if len(answer) == 3:# 如果成功取出前三大的predicts直接指派answer
        predicts.append(answer)
    else: # 如果沒有找到三個的話
        for _ in range(3-len(answer)):
            if len(answer) == 0: # 如果找到0個 則隨機指派
                answer.append(np.random.choice(官方指認欄位))
            else: # 否則建立候選清單(根據已經有的answer隨機挑一筆代入映射表取得tag_list)
                tag_list = mapping[np.random.choice(answer)]
                if len(tag_list) > 0: # 如果候選清單(tag_list)大於0
                    answer.append(np.random.choice(tag_list)) # 從候選清單挑一個候選shop_tag加入answer
                else: # 否則從官方指認欄位隨機挑一個shop_tag 加入answer
                    answer.append(np.random.choice(官方指認欄位))
        predicts.append(answer)
submission = test_data.copy()
submission.iloc[:n,-3:] = np.array(predicts)
submission.head()

100%|██████████| 100/100 [00:10<00:00,  9.83it/s]


,chid,top1,top2,top3
0,10128239,37,25,18
1,10077943,15,36,48
2,10277876,10,15,13
3,10364842,10,15,48
4,10392717,48,15,2
